In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook
import time

In [2]:
data = [(torch.tensor([[0.0000000000, 0.8769030571, 0.0027716185],
          [0.0000000000, 0.8232454658, 0.0027716185],
          [0.0000000000, 0.7695878148, 0.0027716185],
          [0.0000000000, 0.7161158323, 0.0027716185],
          [0.0000000000, 0.6930931807, 0.0027716185],
          [0.0000000000, 0.6700705290, 0.0027716185],
          [0.0000000000, 0.6470478773, 0.0027716185],
          [0.0000000000, 0.6353508830, 0.0027716185],
          [0.0000000000, 0.6234682798, 0.0027716185],
          [0.0000000000, 0.6115856171, 0.0027716185],
          [0.0000000000, 0.6032305956, 0.0027716185],
          [0.0000000000, 0.5946899652, 0.0027716185],
          [0.0000000000, 0.5863349438, 0.0027716185],
          [0.0000000000, 0.5820646286, 0.0027716185],
          [0.0000000000, 0.5779799223, 0.0027716185],
          [0.0000000000, 0.5737096071, 0.0027716185],
          [0.0000000000, 0.6394355893, 0.0027716185],
          [0.0000000000, 0.7049758434, 0.0027716185]]),
  torch.tensor(0.0022172949)),
 (torch.tensor([[0.0000000000, 0.8232454658, 0.0027716185],
          [0.0000000000, 0.7695878148, 0.0027716185],
          [0.0000000000, 0.7161158323, 0.0027716185],
          [0.0000000000, 0.6930931807, 0.0027716185],
          [0.0000000000, 0.6700705290, 0.0027716185],
          [0.0000000000, 0.6470478773, 0.0027716185],
          [0.0000000000, 0.6353508830, 0.0027716185],
          [0.0000000000, 0.6234682798, 0.0027716185],
          [0.0000000000, 0.6115856171, 0.0027716185],
          [0.0000000000, 0.6032305956, 0.0027716185],
          [0.0000000000, 0.5946899652, 0.0027716185],
          [0.0000000000, 0.5863349438, 0.0027716185],
          [0.0000000000, 0.5820646286, 0.0027716185],
          [0.0000000000, 0.5779799223, 0.0027716185],
          [0.0000000000, 0.5737096071, 0.0027716185],
          [0.0000000000, 0.6394355893, 0.0027716185],
          [0.0000000000, 0.7049758434, 0.0027716185],
          [0.0000000000, 0.7707018256, 0.0022172949]]),
  torch.tensor(0.0022172949))]

In [3]:
class LSTMnetwork(nn.Module):
    def __init__(self,input_size=3,hidden_size1=24, hidden_size2=50, hidden_size3=20,output_size=1):
        super().__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.hidden_size3 = hidden_size3
        
        # Add an LSTM and dropout layer:
        self.lstm1 = nn.LSTM(input_size,hidden_size1)
        self.dropout1 = nn.Dropout(p=0.2)
        
        # Add second LSTM and dropout layer:
        self.lstm2 = nn.LSTM(hidden_size1,hidden_size2)
        self.dropout2 = nn.Dropout(p=0.2)
        
        # Add a fully-connected layer:
        self.fc1 = nn.Linear(hidden_size2,hidden_size3)
        
        # Add a fully-connected layer:
        self.fc2 = nn.Linear(hidden_size3,output_size)
        
        # Initialize h0 and c0:
        self.hidden1 = (torch.zeros(1,1,self.hidden_size1),
                       torch.zeros(1,1,self.hidden_size1))
        
        # Initialize h1 and c1:
        self.hidden2 = (torch.zeros(1,1,self.hidden_size2),
                       torch.zeros(1,1,self.hidden_size2))

    def forward(self,seq):
        lstm1_out, self.hidden1 = self.lstm1(seq.view(len(seq),1,-1), self.hidden1)
        dropout1 = self.dropout1(lstm1_out)
        lstm2_out, self.hidden2 = self.lstm2(dropout1.view(len(dropout1),1,-1), self.hidden2)
        dropout2 = self.dropout2(lstm2_out)
        fc1_out = F.relu(self.fc1(dropout2))
        fc2_out = self.fc2(fc1_out)
        return fc2_out[-1]

In [4]:
#model instatiation, loss funciton, optimization
model = LSTMnetwork()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [7]:
for seq, label in data:
        
    #reset the parameters and hidden states
    optimizer.zero_grad()        
    model.hidden1 = (torch.zeros(1,1,model.hidden_size1),
                   torch.zeros(1,1,model.hidden_size1))        

    model.hidden2 = (torch.zeros(1,1,model.hidden_size2),
                   torch.zeros(1,1,model.hidden_size2))

    y_pred_train = model(seq)
    print(y_pred_train)

    loss = criterion(y_pred_train, label.reshape(1,1))
    loss.backward()
    optimizer.step()

tensor([[-0.1760]], grad_fn=<SelectBackward>)
tensor([[-0.1497]], grad_fn=<SelectBackward>)


In [8]:
y_pred_train

tensor([[-0.1497]], grad_fn=<SelectBackward>)

In [25]:
yi = torch.tensor(0.0022)

In [28]:
yi.reshape(1,1)

tensor([[0.0022]])

In [27]:
y_pred_train.size()

torch.Size([1, 1])

In [15]:
for i,j in data: 
    print(i)
    print(j)

tensor([[0.0000, 0.8769, 0.0028],
        [0.0000, 0.8232, 0.0028],
        [0.0000, 0.7696, 0.0028],
        [0.0000, 0.7161, 0.0028],
        [0.0000, 0.6931, 0.0028],
        [0.0000, 0.6701, 0.0028],
        [0.0000, 0.6470, 0.0028],
        [0.0000, 0.6354, 0.0028],
        [0.0000, 0.6235, 0.0028],
        [0.0000, 0.6116, 0.0028],
        [0.0000, 0.6032, 0.0028],
        [0.0000, 0.5947, 0.0028],
        [0.0000, 0.5863, 0.0028],
        [0.0000, 0.5821, 0.0028],
        [0.0000, 0.5780, 0.0028],
        [0.0000, 0.5737, 0.0028],
        [0.0000, 0.6394, 0.0028],
        [0.0000, 0.7050, 0.0028]])
tensor(0.0022)
tensor([[0.0000, 0.8232, 0.0028],
        [0.0000, 0.7696, 0.0028],
        [0.0000, 0.7161, 0.0028],
        [0.0000, 0.6931, 0.0028],
        [0.0000, 0.6701, 0.0028],
        [0.0000, 0.6470, 0.0028],
        [0.0000, 0.6354, 0.0028],
        [0.0000, 0.6235, 0.0028],
        [0.0000, 0.6116, 0.0028],
        [0.0000, 0.6032, 0.0028],
        [0.0000, 0.5947, 0.0028]

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.utils import plot_model
from keras.layers import LSTM, Dense
from keras.layers import Activation, Flatten, Dropout, Concatenate
from keras.optimizers import SGD
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error


In [19]:
#Build Model
inp = Input(shape=(18, 3), name = 'inp')
#build modelA
lstm1A = LSTM(units=24, return_sequences=True, 
               name = 'lstm1A')(inp)
do1A = Dropout(0.2, name = 'do1A')(lstm1A)
lstm2A = LSTM(units=50, name = 'lstm2A')(do1A)
do2A = Dropout(0.2, name = 'do2A')(lstm2A)

#build modelB
lstm1B = LSTM(units=24, return_sequences=True, 
               name = 'lstm1B')(inp)
do1B = Dropout(0.2, name = 'do1B')(lstm1B)
lstm2B = LSTM(units=50, name = 'lstm2B')(do1B)
do2B = Dropout(0.2, name = 'do2B')(lstm2B)

#combine modelA and modelB by dense layer
model_concat = Concatenate()([do2A, do2B])
den1 = Dense(20, activation='relu')(model_concat)
den2 = Dense(2, activation='linear')(den1)

merged = Model(inputs=[inp],outputs=[den2])
plot_model(merged,to_file='demo.png',show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [16]:
merged.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp (InputLayer)                [(None, 18, 3)]      0                                            
__________________________________________________________________________________________________
lstm1A (LSTM)                   (None, 18, 24)       2688        inp[0][0]                        
__________________________________________________________________________________________________
lstm1B (LSTM)                   (None, 18, 24)       2688        inp[0][0]                        
__________________________________________________________________________________________________
do1A (Dropout)                  (None, 18, 24)       0           lstm1A[0][0]                     
_______________________________________________________________________________________